# Scraping of Reddit Comment Data

In [239]:
import praw
r = praw.Reddit( user_agent='Comment Extraction' ,client_id='HjeznSpteYrC4A', client_secret="WVaJyEOw_rf5klZGg-HiQvTV-w0")
r.read_only = True
sub = r.subreddit("iama") #casual conversation,change my view, happy, legal advice, iama
fil = open("comment1.txt", 'a', encoding='utf-8')
key_words = ["i ", "you ", "i'm ", "you're", "im ", "youre "]
submissions = 200
for submis in sub.top('all'):
    comforest = submis.comments
    comforest.replace_more(limit=None)
    for com in comforest:
        bod = com.body.lower()
        for key in key_words:
            if key in bod:
                fil.write(bod + '\n::::::::::::\n')
                break
    submissions -= 1
    if submissions == 0:
        break
fil.close()

KeyboardInterrupt: 

In [2]:
import pandas as pd
import numpy as np
import itertools as it
from collections import Counter
import copy
key_words = ["i ", "you ", "i'm ", "you're", "im ", "youre "]


# Processcing of data to make it useable

In [3]:
the_dat = []
with open("comment1.txt",'r', encoding='utf-8') as f:
    ii = 0
    for key,group in it.groupby(f,lambda line: line.startswith('::::::::::::')):
        if not key:
            the_dat.append([])
            for line in group:
                the_dat[ii].append(line)
            ii += 1
f.close()

In [4]:
for ii in range(len(the_dat)):
    the_dat[ii] = ' '.join(the_dat[ii]).split()

In [5]:
for ii in range(len(the_dat)):
    for yy in range(len(the_dat[ii])):
        the_dat[ii][yy] = the_dat[ii][yy].strip('?').strip(',').strip('!').strip('.')

In [7]:
len(the_dat)

136006

In [8]:
total_words = 0
for comment in the_dat:
    total_words += len(comment)

In [9]:
total_words

6043108

In [10]:
st_words = ['i', 'you', "i'm", "you're"]

# The Corpus with Collocates Generator 

In [11]:
the_corp = {wor : [Counter() for ii in range(5)] for wor in st_words}

In [12]:
for jj in range(len(the_dat)):
    for ii in range(len(the_dat[jj])):
        word = the_dat[jj][ii]
        if word in st_words:
            if word == "im":
                word = "i'm"
            elif word == "youre":
                word = "you're"
            coll = the_dat[jj][ii+1:ii+6]
            for kk in range(len(coll)):
                the_corp[word][kk][coll[kk]] += 1
                
        

In [13]:
for word in the_corp:
    for ii in range(5):
        tot = 0
        ent = the_corp[word][ii]
        for key in ent:
            tot += ent[key]
        ent['TOTAL'] = tot

In [14]:
freq_corp = copy.deepcopy(the_corp)
for k in freq_corp:
    for ii in range(5):
        ent = freq_corp[k][ii]
        tot = ent['TOTAL']
        for key in ent:
            ent[key] /= tot
            ent[key] = round(ent[key] * 100,3)


# The $\LaTeX\ $ table generators 

In [16]:
entry = "i"

In [17]:
for ii in range(5):
    if ii != 0:
        print('&', end='')
    else:
        print("Total" + '&', end='')
    current = the_corp[entry][ii]['TOTAL']
    print("\multicolumn{2}{|c||}{" + str(current) +"}", end='' )
print("\\\\")
print("\\hline")

Total&\multicolumn{2}{|c||}{124663}&\multicolumn{2}{|c||}{123989}&\multicolumn{2}{|c||}{122827}&\multicolumn{2}{|c||}{121088}&\multicolumn{2}{|c||}{119140}\\
\hline


In [18]:
for jj in range(1,21):
    for ii in  range(4):
        if ii != 0:
            print('&', end='')
        else:
            print(str(jj) + '&', end='')
        current = the_corp[entry][ii].most_common(21)[jj]
        curfreq = freq_corp[entry][ii].most_common(21)[jj]
        print(current[0] +'&' + str(curfreq[1]),end='')# str(current[1]) +'&' + str(curfreq[1]),end='')
    print("\\\\")
    print("\\hline")

1&have&6.756&to&6.301&to&6.075&to&3.754\\
\hline
2&am&5.923&a&5.378&a&3.446&a&3.136\\
\hline
3&was&5.735&you&4.436&the&3.235&the&3.108\\
\hline
4&think&4.144&the&2.948&you&2.828&you&2.901\\
\hline
5&know&3.553&that&2.787&that&1.934&i&2.388\\
\hline
6&don't&3.181&it&2.294&i&1.809&and&2.308\\
\hline
7&would&3.136&like&1.958&it&1.557&of&2.104\\
\hline
8&just&2.7&in&1.683&and&1.396&that&1.691\\
\hline
9&can&2.67&have&1.649&my&1.245&in&1.519\\
\hline
10&love&1.982&this&1.613&in&1.238&for&1.471\\
\hline
11&feel&1.818&i&1.544&of&1.08&my&1.302\\
\hline
12&want&1.81&your&1.517&for&1.007&it&1.3\\
\hline
13&hope&1.707&my&1.36&your&1.006&is&1.245\\
\hline
14&do&1.641&think&1.247&is&1.0&your&1.075\\
\hline
15&had&1.506&want&1.152&this&0.996&this&0.934\\
\hline
16&could&1.214&know&1.068&if&0.965&with&0.908\\
\hline
17&can't&1.188&not&1.034&what&0.898&be&0.863\\
\hline
18&really&1.117&wondering&0.925&with&0.776&on&0.802\\
\hline
19&get&1.1&be&0.848&have&0.756&but&0.801\\
\hline
20&will&0.936&for&0.72